# Shen-Castan Edge Detection
Alex Mazur - 10/10/2021

Shen-Castan is an edge detection algorithm outlined in Shen and Castans 1990 paper published in IEEE: *New Edge Detection Methods Based on Exponential Filter* (https://ieeexplore-ieee-org.colorado.idm.oclc.org/stamp/stamp.jsp?tp=&arnumber=118199).
J.R. Parker's book *Algorithms for Image Processing and Computer Vision* was also consulted. Below you will find a notebook that contains a basic-as-possible implementation of Shen and Castans algorithm.

Before we jump in, let's examine what the output of such an algorithm might look like!

<h4><center>Below is the image we start with</center></h4>
<div><img src="track.jpg" width="500"/></div>
<h4><center>Below is the after applying Shen and Castan's smoothing function ISEF</center></h4>
<div><img src="trackISEF.jpg" width="500"/></div>
<h4><center>Below is the image we end up with</center></h4>
<div><img src="trackEdges.jpg" width="500"/></div>

### This algorithm can be broken down into the following steps:
1. Smooth the image using ISEF
2. Compute the band-limited laplacian image (also convert to binary laplacian image where each pixel is a 0 or 1)
3. Find the valid zero crossings
4. Mark the edges that meet our threshold for "edgeness"
5. Run all of these steps together

The following cell contains project wide imports, as well as important constant definitions.

In [168]:
import cv2
import numpy as np

# Constant Values, can impact the resulting image
s_factor = 0.9 # Smoothing factor 0 < sf < 1. Higher values make a "fuzzier" image
window_size = 6 # For computing the adaptive gradient
outline = 25 # For ignoring the edge's of the image
low_thresh, high_thresh = 2, 2 # Edge threshold
do_hysteresis = False
thinFactor = 0

# Helper Functions
def isEdge (row, col, rows, columns):
    if row < outline or row >= rows-outline or col < outline or col >= columns-outline:
        return True
    return False

### Step 1: An implementation of ISEF

In [85]:
# Takes an image (numpy ndarray) and returns an ndarray with ISEF applied
def getISEF (img):
    casual = np.zeros_like (img)
    anti_casual = np.zeros_like (img)
    first_pass = np.zeros_like (img)
    rows, columns = img.shape
    
    b1 = (1.0 - s_factor) / (1.0 + s_factor)
    b2 = s_factor * b1
    
    #################################
    # First compute for rows
    #################################
    
    # Compute boundaries for first and last column
    for col in range (columns):
        casual[0][col] = b1 * img[0][col]
        anti_casual[rows-1][col] = b2 * img[rows-1][col]
    
    # Compute the casual component
    for row in range (1, rows):
        for col in range (columns):
            casual[row][col] = b1 * img[row][col] + s_factor * casual[row-1][col]
    
    # Compute the anti-casual component
    for row in range (rows-2, -1, -1):
        for col in range (columns):
            anti_casual[row][col] = b2 * img[row][col] + s_factor * anti_casual[row+1][col];
    
    # Boundary case
    for col in range (columns-1):
        first_pass[rows-1][col] = casual[rows-1][col];
        
    # Store the sum of casual and anti-casual components in the result
    for row in range (rows-2):
        for col in range (columns-1):
            first_pass[row][col] = casual[row][col] + anti_casual[row+1][col]
            
    #################################
    # Second compute for columns
    #################################
    
    result = first_pass.copy () # Creates a deep copy so we can maintain our first pass results
    
    # Compute boundary conditions
    for row in range (rows):
        casual[row][0] = b1 * first_pass[row][0];
        anti_casual[row][columns-1] = b2 * first_pass[row][columns-1];
        
    # Compute the casual component
    for col in range (1, columns):
        for row in range (rows):
            casual[row][col] = b1 * first_pass[row][col] + s_factor * casual[row][col-1]
            
    # Compute the anti-casual component
    for col in range (columns-2, -1, -1):
        for row in range (rows):
            anti_casual[row][col] = b2 * first_pass[row][col] + s_factor * anti_casual[row][col+1];
    
    # Boundary case
    for row in range (rows):
        result[row][columns-1] = casual[row][columns-1];
        
    # Final computation, sum casual and anti-casual and store in result
    for row in range (rows):
        for col in range (columns-1):
            result[row][col] = casual[row][col] + anti_casual[row][col+1];

    return result

# Note, this could be sped up with parallelism in multiple ways. Since image have constant sizes, you could break the image
#  into x chunks and use x threads to process.
#  ISEF works in two stages, with the second stage relying on the work of the first stage. If there was a thread queue the first
#  stage could push it's work onto the queue while another thread processes the incoming work. This would optimization will be 
#  less than 2x, but it could be usefull to think about. 

### Step 2: Compute the BLI
Shen and Castans paper says that the difference between the original image and the ISEF filtered image is an approximation of the band-limited laplacian image. We can then set all pixels > 0 to 1 to get the final BLI

In [86]:
def getBLI (img, imgISEF):
    rows, columns = img.shape
    result = np.zeros_like (img)
    
    for row in range (rows):
        for col in range (columns):
            result[row][col] = float ((imgISEF[row][col] - img[row][col]) > 0.0); # The inner statement returns bool, cast to float
    return result

### Step 3: Find the zero crossings
This involves an algorithm to determine if an "edge" conforms to the edge constraints from shen-castan. It then applies an adaptive gradient (also from shen-castan). If an edge is not a candidate then that pixel is zeroed.

In [179]:
def isEdgeCandidate (imgBLI, imgISEF, row, col):
    # Positive z - c (row)
    if imgBLI[row][col] == 1.0 and imgBLI[row+1][col] == 0.0:
        return True if (imgISEF[row+1][col] - imgISEF[row-1][col] > 0.0) else False
    
    # Positive z - c (col)
    elif imgBLI[row][col] == 1.0 and imgBLI[row][col+1] == 0.0:
        return True if (imgISEF[row][col+1] - imgISEF[row][col-1] > 0.0) else False
    
    # Negative z - c (row)
    elif imgBLI[row][col] == 1.0 and imgBLI[row-1][col] == 0.0:
        return True if (imgISEF[row+1][col] - imgISEF[row-1][col] < 0.0) else False
    
    # Negative z - c (col)
    elif imgBLI[row][col] == 1.0 and imgBLI[row][col-1] == 0.0:
        return True if (imgISEF[row][col+1] - imgISEF[row][col-1] < 0.0) else False
    
    return False

def adaptiveGradient (imgBLI, imgISEF, row, col):
    sum_on, sum_off = 0.0, 0.0
    num_on, num_off, avg_on, avg_off = 0, 0, 0, 0
    window_start, window_end = int(-window_size/2), int(window_size/2)
    
    for i in range (window_start, window_end):
        for j in range (window_start, window_end):
            if imgBLI[row+i][col+j] > 0.0:
                sum_on += imgISEF[row+i][col+j]
                num_on += 1
            else:
                sum_off += imgISEF[row+i][col+j]
                num_off += 1
                
    avg_off = sum_off / num_off if (sum_off > 0.0) else 0
    avg_on = sum_on / num_on if (sum_on > 0.0) else 0
    
    return avg_off - avg_on

def locateZc (imgISEF, imgBLI):
    rows, columns = imgISEF.shape
    result = np.zeros_like (imgISEF)
    for row in range (rows):
        for col in range (columns):
            if isEdge (row, col, rows, columns):
                continue
            # If there is an edge candidate here then compute the adaptive gradient (shen&castan), else set 0.0
            result[row][col] = adaptiveGradient (imgBLI, imgISEF, row, col) if isEdgeCandidate (imgBLI, imgISEF, row, col) else 0.0
    return result

### Step 4: Find threshold edges
This uses the zero connected image, and the helper function mark_connected, to draw lines of 255 in the image

In [135]:
# This function is used to mark connections, this actually "draws" the lines
edges = None
def mark_connected (row, col, level, img):
    global edges
    if edges[row][col] != 0:
        return 0
    if img[row][col] == 0.0:
        return 0
    if img[row][col] > low_thresh:
        edges[row][col] = 1
    else:
        edges[row][col] = 255
    
    notChainEnd = 0
    notChainEnd |= mark_connected(row, col+1, level+1, img);
    notChainEnd |= mark_connected(row, col-1, level+1, img);
    notChainEnd |= mark_connected(row+1, col+1, level+1, img);
    notChainEnd |= mark_connected(row+1, col, level+1, img);
    notChainEnd |= mark_connected(row+1, col-1, level+1, img);
    notChainEnd |= mark_connected(row-1, col-1, level+1, img);
    notChainEnd |= mark_connected(row-1, col, level+1, img);
    notChainEnd |= mark_connected(row-1, col+1, level+1, img); 
    
    if notChainEnd and level > 0:
        if thinFactor > 0:
            if level % thinFactor != 0:
                edges[row][col] = 255
    
    return 1

def threshold_edges (img):
    global edges
    rows, columns = img.shape
    result = np.zeros_like (img)
    edges = result
    # low_thresh, high_thresh = estimate_thresh (rows, columns)
    if not do_hysteresis:
        low_thresh = high_thresh
    
    for row in range (rows):
        for col in range (columns):
            if isEdge (row, col, rows, columns):
                continue
            if img[row][col] > high_thresh:
                mark_connected (row, col, 0, img)
                
    for row in range (rows):
        for col in range (columns):
            if edges[row][col] == 255:
                edges[row][col] = 0;
    return edges

### Step 5: Finalize the image
Wrap the algorithm driver in a function. This allows for future modifications (such as accounting for color)

In [188]:
def shenCastan (img):
    img = img.astype (float)
    imgISEF = getISEF (img)
    imgBLI = getBLI (img, imgISEF)
    imgZC = locateZc (imgISEF, imgBLI)
    imgEdges = threshold_edges (imgZC)
    
    rows,columns = imgEdges.shape
    for row in range (rows):
        for col in range (columns):
            imgEdges[row][col] = 255 if imgEdges[row][col] > 0 else 0
            
    return imgEdges

### Running the algorithm

In [187]:
img = cv2.imread ("track.jpg", 0)
cv2.imwrite ("trackEdges.jpg", shenCastan (img))

True